**Processing Pipeline 2**

In [2]:
import os
import polars as pl
import pandas as pd
from datetime import datetime
import chardet

In [161]:
input_df = pl.read_csv("data/PP_ALL.csv", encoding="utf-8")
output_df = pl.read_csv("data/Joined_rtf_files.csv", encoding="utf-8")
input_df = input_df.with_columns(pl.col("Datum").str.strptime(pl.Datetime, "%m/%d/%Y %H:%M"))
output_df = output_df.with_columns(pl.col("Datum").str.strptime(pl.Datetime, "%m/%d/%Y %H:%M"))

In [4]:
from datetime import timedelta
def get_first_inputs_before_time(n, output_df_row, input_df, time_window_after_minutes=60):

    #'04/30/2022 18:30'
    target_datetime = dict(zip(output_df.columns, output_df_row))["Datum"]
    start_time = target_datetime.replace(second=0, microsecond=0) - timedelta(minutes=time_window_after_minutes)
    end_time = start_time + 2 * timedelta(minutes=time_window_after_minutes)
    
    filtered = input_df.filter(
        (pl.col("Datum") >= start_time) & (pl.col("Datum") <= end_time)
    )

    def print_all_rows():
        for i in range(len(filtered)):
            print(filtered.row(i))

    if filtered.height > 0:
       #print_all_rows()
       return filtered.sort("Datum").tail(n)
    else:
        return None

row_n = 4
test = get_first_inputs_before_time(5, output_df.row(row_n), input_df)
print("Target row datetime:", dict(zip(output_df.columns, output_df.row(row_n)))["Datum"])
print("\nLast row before target time:", dict(zip(input_df.columns, test))["Datum"])

test_inputs = test.to_pandas()
test_output = output_df.row(row_n)
print("\nTest inputs:")
print(test_inputs)
print("\nTest output:")
print(test_output)


Target row datetime: 2022-04-26 20:00:00

Last row before target time: shape: (5,)
Series: 'Datum' [datetime[μs]]
[
	2022-04-26 19:42:00
	2022-04-26 19:48:00
	2022-04-26 19:53:00
	2022-04-26 20:12:00
	2022-04-26 20:43:00
]

Test inputs:
   LegacyId               Datum                  Operater    A1  \
0   1318759 2022-04-26 19:42:00     DARS Luka Loboda, PIC  NULL   
1   1318760 2022-04-26 19:48:00     DARS Luka Loboda, PIC  NULL   
2   1318761 2022-04-26 19:53:00     DARS Luka Loboda, PIC  NULL   
3   1318762 2022-04-26 20:12:00   DARS Janko Poženel, GNC  NULL   
4   1318763 2022-04-26 20:43:00   DARS Janko Poženel, GNC  NULL   

                                                  B1    C1    A2    B2  \
0  <p><strong>Nesreče</strong></p><p>Na ljubljans...  NULL  None  None   
1  <p><strong>Nesreče</strong></p><p>Na ljubljans...  NULL  None  None   
2  <p><strong>Nesreče</strong></p><p>Na ljubljans...  NULL  None  None   
3  <p><strong>Opozorila</strong></p><p>V sredo, 2...  NULL  None

In [193]:
from datetime import timedelta
def get_first_input_before_time(n, output_df_row, input_df, time_window_after_minutes=0):

    #'04/30/2022 18:30'
    target_datetime = dict(zip(output_df.columns, output_df_row))["Datum"]
    start_time = target_datetime.replace(second=0, microsecond=0) - timedelta(minutes=time_window_after_minutes)
    end_time = start_time + 2 * timedelta(minutes=time_window_after_minutes)
    
    filtered = input_df.filter(
        (pl.col("Datum") <= start_time)
    )

    def print_all_rows():
        for i in range(len(filtered)):
            print(filtered.row(i))

    if filtered.height > 0:
       #print_all_rows()
       return filtered.sort("Datum").tail(n)
    else:
        return None

row_n = 4
test = get_first_input_before_time(1, output_df.row(row_n), input_df)
print("Target row datetime:", dict(zip(output_df.columns, output_df.row(row_n)))["Datum"])
print("\nLast row before target time:", dict(zip(input_df.columns, test))["Datum"])

test_inputs = test.to_pandas()
test_output = output_df.row(row_n)
print("\nTest inputs:")
print(test_inputs)
print("\nTest output:")
print(test_output)


Target row datetime: 2022-04-26 20:00:00

Last row before target time: shape: (1,)
Series: 'Datum' [datetime[μs]]
[
	2022-04-26 19:53:00
]

Test inputs:
   LegacyId               Datum               Operater    A1  \
0   1318761 2022-04-26 19:53:00  DARS Luka Loboda, PIC  NULL   

                                                  B1    C1    A2    B2  \
0  <p><strong>Nesreče</strong></p><p>Na ljubljans...  NULL  None  None   

                                                  C2 TitlePomembnoSLO  ...  \
0  <div><p>Buy vignette for Slovenia online</p><p...             None  ...   

  TitleOvireSLO ContentOvireSLO TitleDeloNaCestiSLO ContentDeloNaCestiSLO  \
0          None            None                None                  None   

  TitleOpozorilaSLO                                ContentOpozorilaSLO  \
0              None  <p>V sredo, 27. 4., bo med 8. in 22. uro, zara...   

  TitleMednarodneInformacijeSLO ContentMednarodneInformacijeSLO  \
0                          None          

In [5]:
from html.parser import HTMLParser
text_example = """<p><strong>Vreme</strong></p><p>Ponekod po Sloveniji megla v pasovih zmanjšuje vidljivost. Prilagodite hitrost!</p><p><strong>Omejitve za tovorna vozila</strong></p><p>Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton:<br>- v soboto, 1. 1., od 8. do 22. ure;<br>- v nedeljo, 2. 1., od 8. do 22. ure.</p><p>Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. <a href="https://www.promet.si/sl/splosne-omejitve" target="_blank">Več.</a></p><p><strong>Dela</strong></p><p>Na primorski avtocesti je ponovno odprt priključek <strong>Črni</strong><strong> Kal </strong>v obe smeri.</p>"""

class HTMLTextExtractor(HTMLParser):
    def __init__(self):
        super().__init__()
        self.text_parts = []

    def handle_data(self, data):
        self.text_parts.append(data)

    def handle_starttag(self, tag, attrs):
        if tag in {"p", "br", "div", "li"}:
            self.text_parts.append(" ")

    def handle_endtag(self, tag):
        if tag in {"p", "br", "div", "li"}:
            self.text_parts.append(" ")

    def get_text(self):
        # Join and normalize whitespace
        return ' '.join(''.join(self.text_parts).split())

def strip_html(html):
    parser = HTMLTextExtractor()
    parser.feed(html)
    return parser.get_text()

# Example usage
cleaned_text = strip_html(text_example)
print("\nCleaned text:")


Cleaned text:


In [6]:
from bs4 import BeautifulSoup
from collections import Counter

input_df = input_df
# html is in column "B1"
html = """
<p><strong>Vreme</strong></p><p>Ponekod po Sloveniji megla v pasovih zmanjšuje vidljivost. Prilagodite hitrost!</p><p><strong>Omejitve za tovorna vozila</strong></p><p>Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton:<br>- v soboto, 1. 1., od 8. do 22. ure;<br>- v nedeljo, 2. 1., od 8. do 22. ure.</p><p>Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. <a href="https://www.promet.si/sl/splosne-omejitve" target="_blank">Več.</a></p><p><strong>Dela</strong></p><p>Na primorski avtocesti je ponovno odprt priključek <strong>Črni</strong><strong> Kal </strong>v obe smeri.</p>
"""

# Counter to collect tag frequencies
tag_counter = Counter()

# Loop through each HTML string
for html in input_df["B1"]:
    soup = BeautifulSoup(html, "html.parser")
    for tag in soup.find_all(True):
        tag_counter[tag.name] += 1  # Count each tag once per appearance

# Print result
print("Tag counts:")
for tag, count in tag_counter.items():
    print(f"<{tag}>: {count}")


Tag counts:
<p>: 1983381
<strong>: 1533299
<br>: 611357
<a>: 217990
<u>: 16373
<em>: 326
<ul>: 69
<li>: 84


In [ ]:
# Count contents of <strong> tags
strong_text_counter = Counter()

for html in input_df["B1"]:
    soup = BeautifulSoup(html, "html.parser")
    for tag in soup.find_all("strong"):
        text = tag.get_text(strip=True)
        if text:
            strong_text_counter[text] += 1

# Use most_common() to sort by count descending
sorted_counts = strong_text_counter.most_common()

# Print sorted results
print("Counts of <strong> tag contents (sorted):")
for text, count in sorted_counts:
    print(f"{text}: {count}")

In [ ]:

html = """
<p><strong>Vreme</strong></p><p>Ponekod po Sloveniji megla v pasovih zmanjšuje vidljivost. Prilagodite hitrost!</p><p><strong>Omejitve za tovorna vozila</strong></p><p>Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton:<br>- v soboto, 1. 1., od 8. do 22. ure;<br>- v nedeljo, 2. 1., od 8. do 22. ure.</p><p>Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. <a href="https://www.promet.si/sl/splosne-omejitve" target="_blank">Več.</a></p><p><strong>Dela</strong></p><p>Na primorski avtocesti je ponovno odprt priključek <strong>Črni</strong><strong> Kal </strong>v obe smeri.</p>
"""

json_format = {
    
    "Delo na cesti:": "",
    "Zastoji:": "",
    "Ovire:": "",
    "Nesreče:": "",
    "Vreme:": "",
    "Prireditve:": "",
    "Opozorila:": "",
    "Mejni prehodi:": "",
    "Zimske razmere:":"",
    "Dela:":"",
    "Popolne zapore:":"",
    "leftover_html": ""
}


In [28]:
from bs4 import BeautifulSoup
import re
import json
from IPython.display import display, HTML

# Template
json_format = {
    "Delo na cesti": "",
    "Dela": "",
    "Delovne zapore": "",
    "Popolne zapore": "",
    "Zastoji": "",
    "Ovire": "",
    "Nesreče": "",
    "Vreme": "",
    "Opozorila": "",
    "Zimske razmere": "",
    "Prireditve": "",
    "Mejni prehodi": "",
    "Omejitve za tovorna vozila": "",
    "leftover_html": ""
}

def parse_html_into_json(html, json_format, timestamp=None):
    # Normalize keys for matching
    normalized_keys = {
        re.sub(r":+", "", k).strip().lower(): k
        for k in json_format if k != "leftover_html"
    }

    soup = BeautifulSoup(html, "html.parser")
    elements = soup.find_all(["p", "br", "a"])

    current_heading = None
    content_map = {k: "" for k in json_format if k != "leftover_html"}
    leftover = []

    for tag in elements:
        text = tag.get_text(separator=" ", strip=True)

        # Skip empty elements
        if not text:
            continue

        # Check if tag starts with a known section heading
        for normalized_heading, real_key in normalized_keys.items():
            if text.lower().startswith(normalized_heading):
                current_heading = real_key
                # Remove the heading from the text itself
                cleaned = re.sub(f"^{re.escape(normalized_heading)}:?\s*", "", text, flags=re.I)
                if cleaned:
                    content_map[real_key] += cleaned + " "
                break
        else:
            if current_heading:
                content_map[current_heading] += text + " "
            else:
                leftover.append(text)

    # Strip trailing whitespace
    for key in content_map:
        content_map[key] = content_map[key].strip()

    json_format.update(content_map)
    json_format["leftover_html"] = "\n".join(leftover).strip()
    if timestamp:
        json_format["timestamp"] = timestamp.strftime("%d-%m-%Y %H:%M:%S")
    return json_format

    

# Call the function to parse the HTML and fill the json_format
row = input_df.select(["B1", "Datum"]).sample(1).row(0)
random_html, timestamp = row

print("Random HTML content from DataFrame:")
display(HTML(random_html))

myjson=parse_html_into_json(random_html, json_format, timestamp=timestamp)
print(json.dumps(myjson, indent=2, ensure_ascii=False))


Random HTML content from DataFrame:


{
  "Delo na cesti": "Glavna cesta Slovenska Bistrica - Ptuj je ponovno odprta.",
  "Dela": "",
  "Delovne zapore": "",
  "Popolne zapore": "- Na dolenjski avtocesti bo do 5. 12. zaprt priključek Dobruška vas, izvoz iz smeri Obrežja in uvoz proti Ljubljani. - Na gorenjski avtocesti bo do 2. decembra zaprt priključek Radovljica, izvoz iz smeri Jesenic in uvoz proti Ljubljani. Zaprto je tudi počivališče. - Na štajerski avtocesti bosta predvidoma do 28. novembra zaprta izvoz in uvoz Maribor sever proti Šentilju. - Do 2. 12. bo zaprta cesta Žuniči - Vinica , med Preloko in Balkovci. Napovedane zapore cest: - Na dolenjski avtocesti bo od četrtka, 24. 11., do 5. decembra zaprt priključek Dobruška vas proti Ljubljani . - Od sobote, 26. 11. od 6. ure, do nedelje 27. 11. do 10. ure, bo zaprta primorska avtocesta med razcepom Srmin in priključkom Črni Kal proti Ljubljani . - Cesta Podpeč - Ig bo zaprta od sobote, 26. 11. od 7. ure, do nedelje, 27. 11. do 21. ure. - Cesta Žirovnica - Begunje bo v

In [ ]:

columns = ["Datum", "A1", "B1", "C1","ContentPomembnoSLO", "ContentNesreceSLO" , "ContentNesreceSLO", "ContentVremeSLO", "ContentOvireSLO ", "ContentDeloNaCestiSLO", "ContentOpozorilaSLO"]
semantic_compare = {
  "Delo na cesti": "ContentDeloNaCestiSLO",
  "Zastoji": "",
  "Ovire": "ContentOvireSLO",
  "Nesreče": "ContentNesreceSLO",
  "Vreme": "ContentVremeSLO",
  "Prireditve": "",
  "Opozorila": "ContentOpozorilaSLO",
  "Mejni prehodi": "",
  "Delovne zapore": "",
  "Zimske razmere": "",
  "Dela": "",
  "Popolne zapore": "",
  "Omejitve za tovorna vozila": "",
  "leftover_html": "ContentPomembnoSLO"
}
# Call the function to parse the HTML and fill the json_format
#random_html = input_df["B1"].sample(1).item()  # Randomly sample one HTML entry from the DataFrame
#print("Random HTML content from DataFrame:")
#display(HTML(random_html))

#myjson=parse_html_into_json(random_html, json_format)
#print(json.dumps(myjson, indent=2, ensure_ascii=False))
    

In [ ]:
from difflib import SequenceMatcher

def similarity(a, b):
    return SequenceMatcher(None, a.strip(), b.strip()).ratio()

# Sample the row (you already sampled the HTML)
row = input_df.sample(1)
random_html = row["B1"].item()
parsed = parse_html_into_json(random_html, json_format.copy())  # use copy to avoid overwriting

print("=== Semantic Comparison ===")
for key, column in semantic_compare.items():
    if not column:
        continue  # skip if no mapped column

    parsed_value = (parsed.get(key) or "").strip()
    actual_value = (row[column].item() if column in row else "") or ""
    actual_value = actual_value.strip()

    match_ratio = similarity(parsed_value, actual_value)
    
    if match_ratio < 0.8:
        print(f"❗️ Low similarity for key: {key} (ratio: {match_ratio:.2f})")

    print(f"🧠 {key} → {column}")
    print(f"✅ Similarity: {match_ratio:.2f}")
    print(f"📤 Extracted:\n{parsed_value[:300]}{'...' if len(parsed_value) > 300 else ''}")
    print(f"📥 Actual:\n{actual_value[:300]}{'...' if len(actual_value) > 300 else ''}")
    print("-" * 80)


=== Semantic Comparison ===
❗️ Low similarity for key: Delo na cesti (ratio: 0.78)
🧠 Delo na cesti → ContentDeloNaCestiSLO
✅ Similarity: 0.78
📤 Extracted:
Več o delovnih zaporah v prometni napovedi . Več o delovnih zaporah v prometni napovedi Na južni ljubljanski obvoznici bo zaradi rekonstrukcije zaprt priključek Ljubljana Vič v obe smeri od srede,16. 8., do 25. 8. 2023. Od nedelje do četrtka, 31. 8., bo na severni ljubljanski obvoznici zaprt uvoz iz...
📥 Actual:
<p><a href="https://www.promet.si/sl/aktualna-napoved" target="_blank">Več o delovnih zaporah v prometni napovedi</a>.</p><p><strong>Na južni ljubljanski obvoznici </strong>bo zaradi rekonstrukcije zaprt priključek Ljubljana Vič v obe smeri od srede,16. 8., do 25. 8. 2023.</p><p>Od nedelje do četrtk...
--------------------------------------------------------------------------------
🧠 Ovire → ContentOvireSLO
✅ Similarity: 0.96
📤 Extracted:
Na gorenjski avtocesti je pred galerijo Moste proti Karavankam zaprt vozni pas, okvara v

We now have a json structure for the data. now we need to compare the json input and the gennerated output to see if they match.

```python

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained sentence embedding model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

def get_similarity_score(output_message, json_string):
    # Compute embeddings
    embeddings = model.encode([output_message, json_string], convert_to_tensor=True)

    # Compute cosine similarity
    similarity_score = util.cos_sim(embeddings[0], embeddings[1])
    
    return similarity_score.item()


def parse_output_messages(output_row, columns):
    row_dict = dict(zip(columns, output_row))
    message = ""
    for column in ["content_01", "content_02", "content_03", "content_04", "content_05"]:
        if row_dict.get(column) is not None:
            if column == "content_05":
                if '\x00' == row_dict[column]:
                    continue
            message += f"{row_dict[column]} "
    return message.strip()

from tqdm import tqdm

def compare_json_to_output(input_df, output_df, n_examples=1, n_samples=5):
    traffic_examples = pd.DataFrame(columns=["Output Message", "Input_Message", "Input JSON", "Datum", "Similarity Score"])
    print(f"Comparing {n_examples} examples from output_df with input_df...")

    for i in tqdm(range(n_examples), desc="Examples"):  # outer loop progress bar
        try:
            traffic_examples_specific = pd.DataFrame(columns=["Output Message", "Input_Message", "Input JSON", "Datum", "Similarity Score"])
            timestamps = []
            similarity_scores = []
            timestamp_scores_dict = {}
            output_df_row = output_df.sample(1).row(0)  # Randomly sample one row from output_df
            input_rows = get_first_inputs_before_time(n_samples, output_df_row, input_df)

            # Inner loop over input rows with tqdm progress bar
            for _, input_row in tqdm(input_rows.to_pandas().iterrows(), total=len(input_rows), desc=f"Samples for example {i+1}", leave=False):
                try:
                
                    timestamps.append(input_row["Datum"])
                    html = input_row["B1"]
                    if isinstance(html, pl.Series):  # if it's a Polars Series, get the first item
                        html = html[0]
                    html = str(html) 
                    myjson = parse_html_into_json(html, json_format)
                    output = parse_output_messages(output_df_row, output_df.columns)
                    
                    full_string = ""
                    for key, value in myjson.items():
                        if value != "":
                            full_string += f"{value} "
                            
                    similarity_score = get_similarity_score(output, full_string)
                    # print(f"Similarity score: {similarity_score:.4f}")
                    similarity_scores.append(similarity_score)
                    timestamp_scores_dict[input_row["Datum"]] = similarity_score
                    
                    new_row = {
                        "Output Message": output,
                        "Input_Message": full_string,
                        "Input JSON": json.dumps(myjson, indent=2, ensure_ascii=False),
                        "Datum": input_row["Datum"],
                        "Similarity Score": similarity_score
                    }

                    new_df = pd.DataFrame([new_row])
                    traffic_examples_specific = pd.concat([traffic_examples_specific, new_df], ignore_index=True)

                except Exception as e:
                    print(f"Error processing input row: {e}")
                    continue
            
            # print(f"Average similarity score for example {i+1}: {sum(similarity_scores) / len(similarity_scores):.4f}")
            # print(similarity_scores)
            
            output_timestamp = output_df_row[0]  # or 0 — depends on your schema
            
            # print("*" * 80)
            # print(f"Output timestamp: {output_timestamp}")

            converted_dict = {str(k): v for k, v in timestamp_scores_dict.items()}
            # print(json.dumps(converted_dict, indent=2, ensure_ascii=False))
            
            best_example = traffic_examples_specific.loc[traffic_examples_specific["Similarity Score"].idxmax()]
            # print(f"Best example for output row {i+1}:", best_example["Similarity Score"])
            traffic_examples = pd.concat([traffic_examples, best_example.to_frame().T], ignore_index=True)
            
        except Exception as e:
            print(f"Error calculating average similarity score for example {i+1}: {e}")
            continue
        
    return traffic_examples

traffic_examples = compare_json_to_output(input_df, output_df, n_examples=len(output_df), n_samples=15)
# print("\nTraffic Examples DataFrame:")
# print(traffic_examples.head())

traffic_examples.to_csv("trainingdataset3.csv", index=False, encoding="utf-8")
print("\nTraffic examples saved to 'trainingdataset3.csv'.")


In [214]:
#optimizzed version
import pandas as pd
import polars as pl
import json
from tqdm import tqdm
import warnings
from sentence_transformers import SentenceTransformer, util

warnings.filterwarnings("ignore")

# Assume these functions are defined elsewhere in your script
# from your_utils import parse_html_into_json, get_first_inputs_before_time

# It's good practice to load the model once
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')


def parse_output_messages(output_row, columns):
    # This function is fine as is, but we'll call it more efficiently
    # Assuming output_row is a tuple or list from iter_rows()
    row_dict = dict(zip(columns, output_row))
    message_parts = []
    for i in range(1, 6): # A slightly cleaner way to loop through content_01 to content_05
        col_name = f"content_0{i}"
        content = row_dict.get(col_name)
        if content and content != '\x00':
            message_parts.append(str(content))
    return " ".join(message_parts)


def compare_json_to_output_optimized(input_df, output_df, n_samples=15):
    """
    Optimized function to find the best input match for each output message.
    """
    best_results_list = []
    output_columns = output_df.columns

    print(f"Comparing {len(output_df)} output examples with {n_samples} input samples each...")

    # Iterate directly over output_df rows. This is much cleaner than sampling in a loop.
    for output_row_tuple in tqdm(output_df.iter_rows(), total=len(output_df), desc="Processing Output Examples"):

        try:
            # --- 1. Process the output ONCE per outer loop ---
            output_message = parse_output_messages(output_row_tuple, output_columns)
            if not output_message: # Skip if the output message is empty
                continue
            
            # Encode the single output message
            output_embedding = model.encode(output_message, convert_to_tensor=True)

            # --- 2. Get candidate inputs ---
            input_rows_df = get_first_inputs_before_time(n_samples, output_row_tuple, input_df).to_pandas()
            if input_rows_df.empty:
                continue

            # --- 3. Prepare all input texts for BATCH processing ---
            input_texts_to_encode = []
            processed_inputs = [] # Keep track of the full parsed data

            for _, input_row in input_rows_df.iterrows():
                html = str(input_row.get("B1", ""))
                
                # Assume parse_html_into_json is defined and returns a dict
                myjson = parse_html_into_json(html, json_format) 
                
                full_string = " ".join(value for value in myjson.values() if value)
                input_texts_to_encode.append(full_string)
                processed_inputs.append({
                    "Input_Message": full_string,
                    "Input_JSON_str": json.dumps(myjson, indent=2, ensure_ascii=False),
                    "Datum": input_row["Datum"]
                })

            # --- 4. BATCH encode all input texts at once ---
            input_embeddings = model.encode(input_texts_to_encode, convert_to_tensor=True)
            
            # --- 5. Compute all similarities in a single, fast operation ---
            cosine_scores = util.cos_sim(output_embedding, input_embeddings)
            # print(f"Cosine scores: {cosine_scores}")
            
            # Find the index of the best score
            best_sample_idx = cosine_scores[0].argmax().item()

            # --- 6. Select the best result ---
            best_input_data = processed_inputs[best_sample_idx]
            best_score = cosine_scores[0][best_sample_idx].item()
            # print(f"Best score for output row {output_row_tuple[0]}: {best_score:.4f}")
            
            # Append the single best result for this output to our list
            best_results_list.append({
                "Output Message": output_message,
                "Input_Message": best_input_data["Input_Message"],
                "Input JSON": best_input_data["Input_JSON_str"],
                "Datum": best_input_data["Datum"],
                "Similarity Score": best_score
            })

        except Exception as e:
            # It's good to know which row caused an error
            output_timestamp = output_row_tuple[0] 
            print(f"Error processing output row for timestamp {output_timestamp}: {e}")
            continue

    # --- 7. Create the final DataFrame ONCE from the list of results ---
    if not best_results_list:
        print("No valid examples were processed.")
        return pd.DataFrame()
        
    final_df = pd.DataFrame(best_results_list)
    return final_df

# Assuming input_df and output_df are already loaded as Polars DataFrames
# And json_format is defined
#sample the outputdf  
output_df_sample = output_df.sample(n=100)  # Sample 15 rows for testing
traffic_examples = compare_json_to_output_optimized(input_df, output_df_sample, n_samples=20)
traffic_examples.to_csv("trainingdataset_optimized.csv", index=False, encoding="utf-8")
# print("\nOptimized traffic examples saved to 'trainingdataset_optimized.csv'.")

Comparing 100 output examples with 20 input samples each...


Processing Output Examples:  21%|██        | 21/100 [00:05<00:14,  5.64it/s]

Error processing output row for timestamp 2022-07-23 18:30:00: 'NoneType' object has no attribute 'to_pandas'


Processing Output Examples:  78%|███████▊  | 78/100 [00:21<00:05,  3.67it/s]


KeyboardInterrupt: 

Now we hhave a system that checks the output of the model against the concated json structure. And we can comare which specific innput matches the output the most. We will match the output to the input by comparing the output to the input and finding the closest match. and recording the similarity score and json structure along with the input and output in the training dataframe. 

B1  conntains tags like  vreme nesreče ovire.... whic is also i separate columns. Now tis information is often duplicated and i need to join it. 

In [ ]:
print("\nTest inputs:")
print(test_inputs.iloc[0])

def parse_input_message(input_row, columns):
    row_dict = dict(zip(columns, input_row))
    message = ""
    for column in ["Datum", "A1", "B1", "C1","ContentPomembnoSLO" , "ContentNesreceSLO", "ContentVremeSLO", "ContentOvireSLO ", "ContentDeloNaCestiSLO", "ContentOpozorilaSLO"]:
        value = row_dict.get(column)
        if value is not None and value != "NULL":
            if isinstance(value, datetime):
                clean_value = value.strftime("%Y-%m-%d %H:%M:%S")
            elif isinstance(value, str):
                clean_value = strip_html(value)
                #new line 
            message += f"{column}: {clean_value} "
    #print("Input message:", message.strip())
    return message.strip()
parse_input_message(test_inputs.iloc[0], test_inputs.columns)



Test inputs:
LegacyId                                                                     1318757
Datum                                                            2022-04-26 19:33:00
Operater                                                       DARS Luka Loboda, PIC
A1                                                                              NULL
B1                                 <p><strong>Nesreče</strong></p><p>Na ljubljans...
C1                                                                              NULL
A2                                                                              None
B2                                                                              None
C2                                 <div><p>Buy vignette for Slovenia online</p><p...
TitlePomembnoSLO                                                                None
ContentPomembnoSLO                                                              None
TitleNesreceSLO                                    

'Datum: 2022-04-26 19:33:00 B1: Nesreče Na ljubljanski severni obvoznici zaradi prometne nesreče med krožiščem Tomačevo in Bežigradom promet proti Kosezam poteka samo po enem prometnem pasu. Ovire Na štajerski avtocesti je zaradi živali zaprt prehitevalni pas med Šentjakobom in Zadobrovo proti Ljubljani. Opozorila V sredo, 27. 4., bo med 8. in 22. uro, zaradi praznika veljala splošna omejitev prometa tovornih vozil, katerih največja dovoljena masa presega 7,5 t. ContentNesreceSLO: Na ljubljanski severni obvoznici zaradi prometne nesreče med krožiščem Tomačevo in Bežigradom promet proti Kosezam poteka samo po enem prometnem pasu. ContentNesreceSLO: Na ljubljanski severni obvoznici zaradi prometne nesreče med krožiščem Tomačevo in Bežigradom promet proti Kosezam poteka samo po enem prometnem pasu. ContentOpozorilaSLO: V sredo, 27. 4., bo med 8. in 22. uro, zaradi praznika veljala splošna omejitev prometa tovornih vozil, katerih največja dovoljena masa presega 7,5 t.'

In [17]:
print("\nTest output:")
print(test_output)
def parse_output_messages(output_row, columns):
    row_dict = dict(zip(columns, output_row))
    message = ""
    for column in ["content_01", "content_02", "content_03", "content_04", "content_05"]:
        if row_dict.get(column) is not None:
            if column == "content_05":
                if '\x00' == row_dict[column]:
                    continue
            message += f"{row_dict[column]} "
    return message.strip()


parse_output_messages(test_output, output_df.columns)


Test output:
(datetime.datetime(2022, 4, 26, 20, 0), 'TMP-102.rtf', 'April 2022', 'Na južni ljubljanski obvoznici je zaradi pokvarjenega vozila  zaprt izvoz Ljubljana - jug proti dolenjski avtocesti.', 'Zaradi del je na severni ljubljanski obvoznici zaprt uvoz Nove Jarše proti štajerski avtocesti.', 'Na cesti Rogatec - Dobovec poteka promet zaradi prometne nesreče izmenično enosmerno.', 'Na Obrežju je povečana promet osebnih vozil pri vstopu v državo, v Jelšanah pa pri izstopu iz nje.', '\x00')


'Na južni ljubljanski obvoznici je zaradi pokvarjenega vozila  zaprt izvoz Ljubljana - jug proti dolenjski avtocesti. Zaradi del je na severni ljubljanski obvoznici zaprt uvoz Nove Jarše proti štajerski avtocesti. Na cesti Rogatec - Dobovec poteka promet zaradi prometne nesreče izmenično enosmerno. Na Obrežju je povečana promet osebnih vozil pri vstopu v državo, v Jelšanah pa pri izstopu iz nje.'

Okay now we have a way to smeantically check the output of the model against the input json structure. But we still need a parser that will check if events match. I believe that semantic similarity is not enough inn this case. We need to check if the events match. also maybe just count the number of events that match and dont match. Create a parser thhat will be able to identify 80% of the events that match and 20% that dont match. I dont need to go into to much detail here, just create a parser that will be able to identify the events that match and the ones that dont match.

In [ ]:
import re
from enum import Enum
from typing import List, Optional, Dict, Any
from datetime import datetime
from pydantic import BaseModel, Field

json_thingy = {
  "Delo na cesti": "ContentDeloNaCestiSLO",
  "Zastoji": "",
  "Ovire": "ContentOvireSLO",
  "Nesreče": "ContentNesreceSLO",
  "Vreme": "ContentVremeSLO",
  "Prireditve": "",
  "Opozorila": "ContentOpozorilaSLO",
  "Mejni prehodi": "",
  "Delovne zapore": "",
  "Zimske razmere": "",
  "Dela": "",
  "Popolne zapore": "",
  "Omejitve za tovorna vozila": "",
  "leftover_html": "ContentPomembnoSLO"
}

class EventType(str, Enum):
    ROADWORK = "Delo na cesti"                      
    CONGESTION = "Zastoji"                   
    OBSTACLE = "Ovire"                       
    ACCIDENT = "Nesreče"                    
    WEATHER = "Vreme"
    EVENT = "Prireditve"                     
    WARNING = "Opozorila"                    
    BORDER_CROSSING = "Mejni prehodi"         
    ROAD_CLOSURE = "Delovne zapore"         
    WINTER_CONDITIONS = "Zimske razmere"       
    WORK = "Dela"                            
    FULL_CLOSURE = "Popolne zapore"          
    TRUCK_RESTRICTION = "Omejitve za tovorna vozila"   
    UNKNOWN = "unknown"                         

class ParsedTrafficEvent(BaseModel):
    """
    A simplified, practical model representing a single parsed traffic event.
    """
    event_type: EventType = Field(..., description="The classified type of the event.")
    location_description: str = Field(..., description="A description of the road or area affected.")
    
    delay_minutes: Optional[int] = Field(None, description="Estimated delay in minutes.")
    detour_available: Optional[bool] = Field(None, description="Indicates if a detour is mentioned.")
    
    source_text: str = Field(..., description="The original text segment this event was parsed from.")
    parse_timestamp: datetime = Field(default_factory=datetime.now)

# --- 2. Keyword and Regex Definitions (The "Brain" of the Parser) ---

# Map keywords to an EventType. Order matters: more specific keywords should come first.
# We use word stems to catch variations (e.g., "dela", "delovnih").
EVENT_TYPE_KEYWORDS: Dict[EventType, List[str]] = {
    EventType.TRUCK_RESTRICTION: ["omejitev prometa tovornih", "tovorna vozila"],
    EventType.ACCIDENT: ["nesreč", "nesr"],
    EventType.ROADWORK: ["del na cesti", "delovna zapora", "delovnih zaporah", "montažni most"],
    EventType.CONGESTION: ["zastoj", "zamuda", "promet upočasnjen", "kolona", "zgostitev prometa"],
    EventType.OBSTACLE: ["ovira", "predmet na cesti", "razsut tovor", "žival"],
    EventType.WEATHER: ["vremensk", "burja", "sneg", "poledic", "megla", "vrem", "piha"],
}

def _find_event_type(text: str) -> EventType:
    """Finds the event type based on keywords in the text."""
    lower_text = text.lower()
    for event_type, keywords in EVENT_TYPE_KEYWORDS.items():
        if any(keyword in lower_text for keyword in keywords):
            return event_type
    return EventType.UNKNOWN

def _extract_delay(text: str) -> Optional[int]:
    """Extracts delay time in minutes. Averages ranges."""
    # Matches patterns like "zamuda 5 - 10 minut", "zamuda 10 minut"
    match = re.search(r"zamuda\s*(\d+)(?:\s*-\s*(\d+))?\s*minut", text.lower())
    if match:
        start_delay = int(match.group(1))
        end_delay = int(match.group(2)) if match.group(2) else start_delay
        return (start_delay + end_delay) // 2
    return None

def _extract_location(text: str) -> str:
    """A simple location extractor. It cleans up some common clutter."""
    # This can be improved, but for an 80/20 solution, we remove known non-location phrases.
    location = text.strip()
    clutter = [
        "Več o delovnih zaporah v prometni napovedi",
        ", zamuda 5 - 10 minut",
        ", zamuda 10 minut",
        "."
    ]
    for c in clutter:
        location = location.replace(c, "").strip()
    return location

# --- 3. The Main Parsing Function ---

def parse_traffic_text(text: str) -> List[ParsedTrafficEvent]:
    """
    Parses a block of unstructured traffic text into a list of structured event objects.

    Args:
        text (str): The raw input text containing multiple traffic events.

    Returns:
        List[ParsedTrafficEvent]: A list of parsed event objects.
    """
    events: List[ParsedTrafficEvent] = []
    
    # Split text into lines. This is a robust starting point for traffic reports.
    # We filter out empty lines.
    lines = [line.strip() for line in text.split('.') if line.strip()]

    for line in lines:
        event_type = _find_event_type(line)

        # We only create an event if it's something we can classify.
        # This ignores generic lines like "Več o..." if they don't contain other keywords.
        if event_type != EventType.UNKNOWN:
            delay = _extract_delay(line)
            detour = "obvoz" in line.lower()
            location = _extract_location(line)

            event = ParsedTrafficEvent(
                event_type=event_type,
                location_description=location,
                delay_minutes=delay,
                detour_available=detour if detour else None, # Only set if True
                source_text=line
            )
            events.append(event)
    
    return events


# --- 4. Example Usage ---
if __name__ == "__main__":
    
    df = pd.read_csv("trainingdataset_optimized.csv", encoding="utf-8")
    sample = df.sample(n=1)  # Sample one row for testing
    input_text = sample["Input_Message"].iloc[0] # Get the first input message for testing
    output_text = sample["Output Message"].iloc[0] # Get the first output message for testing

    print("--- Parsing Input Text ---")
    print(input_text)  # Display the input text
    parsed_events = parse_traffic_text(input_text)
    
    print("--- Parsing Output Text ---")
    print(output_text)  # Display the output text
    parsed_events2 = parse_traffic_text(output_text)  # Parse both input and output texts

    print(f"\nFound {len(parsed_events)}/{len(parsed_events2)} events")
    print("-" * 25)
    
    

    for i, event in enumerate(parsed_events):
        print(f"Event #{i+1}:")
        # .model_dump_json is a Pydantic v2 method, for v1 use .json()
        try:
            print(event.model_dump_json(indent=2, exclude={'parse_timestamp'}))
        except AttributeError:
            print(event.json(indent=2, exclude={'parse_timestamp'})) # For Pydantic v1
        print("-" * 25)
        
    for i, event in enumerate(parsed_events2):
        print(f"Event #{i+1}:")
        # .model_dump_json is a Pydantic v2 method, for v1 use .json()
        try:
            print(event.model_dump_json(indent=2, exclude={'parse_timestamp'}))
        except AttributeError:
            print(event.json(indent=2, exclude={'parse_timestamp'}))

--- Parsing Input Text ---
Na gorenjski avtocesti bo promet skozi predor Karavanke potekal izmenično enosmerno danes in jutri med 8. in 16. uro. Na podravski avtocesti med Podlehnikom in Gruškovjem v obe smeri bo v četrtek, 9. 3. med 12. in 12.30, popolna zapora zaradi minerskih del v kamnolomu Zakl. Na vipavski hitri cesti je zaprt priključek Šempeter v obe smeri. Cesta Litija - Zagorje bo zaprta pri Šklendrovcu vsako soboto med 7:30 in 13:30 do septembra 2023. Več o delovnih zaporah v prometni napovedi. Več o delovnih zaporah v prometni napovedi. Cesta čez prelaz Vršič je prevozna samo za osebna vozila z verigami.
--- Parsing Output Text ---
Na vseh vpadnicah v Ljubljano je oviran promet zaradi nedelujočih semaforjev. Nastajajo zastoji. Na štajerski avtocesti proti Mariboru je obtičal pokvarjen tovornjak na izvozu Trojane. Zaradi del poteka promet skozi predor Karavanke izmenično enosmerno.


Found 2/2 events
-------------------------
Event #1:
{
  "event_type": "Delo na cesti",
  "l

im kind of thinking what is the point of this, why do i have to transform the input and the output into some event formats? liek this 
Event #2:
{
  "event_type": "Delo na cesti",
  "location_description": "",
  "delay_minutes": null,
  "detour_available": null,
  "source_text": "Več o delovnih zaporah v prometni napovedi"
}

when i could have full jsons 
like 
json_format = {
  "Delo na cesti": "ContentDeloNaCestiSLO",
  "Zastoji": "",
  "Ovire": "ContentOvireSLO",
  "Nesreče": "ContentNesreceSLO",
  "Vreme": "ContentVremeSLO",
  "Prireditve": "",
  "Opozorila": "ContentOpozorilaSLO",
  "Mejni prehodi": "",
  "Delovne zapore": "",
  "Zimske razmere": "",
  "Dela": "",
  "Popolne zapore": "",
  "Omejitve za tovorna vozila": "",
  "leftover_html": "ContentPomembnoSLO"
}

i goess that the point is that im trying to match the input to be as close as possible to the output. I can not change the output but i can customize the inputs to some degree. so i think i should create aprser for the outputs so that i get the json format of the output and comare it with the json format of the input which is already correctly structured. and i will see if i have to cut anything from the inputs to match the whole thing better. beacauset his is  nlp model fine tuning not a person reading and thinking whats important based on 3 h of past information and making shit up the fly. No what i want is 100 matching input that then gets put in a public speaking form . So the fact that there is maybe a different information 3 0min before does not matter to me what matter is the information that was actually used. and the end result here is that im hoping for a higher semantic match. of I/O. 


**Mapping output for  json_format**

In [15]:
json_format = {
    "Delo na cesti": "",
    "Dela": "",
    "Delovne zapore": "",
    "Popolne zapore": "",
    "Zastoji": "",
    "Ovire": "",
    "Nesreče": "",
    "Opozorila": "",
    "Vreme": "",
    "Zimske razmere": "",
    "Prireditve": "",
    "Mejni prehodi": "",
    "Omejitve za tovorna vozila": "",
    "leftover_html": ""
}

In [16]:
output_df = output_df.to_pandas()
output_df.head(3)

,Datum,TMP_file_name,TMP_folder_name,content_01,content_02,content_03,content_04,content_05
0,2022-04-30 18:30:00,TMP-1.rtf,April 2022,Zaradi prometne nesreče je zaprta regionalna c...,Na mejnem prehodu Obrežje vozniki na vstop v d...,None,None,None
1,2022-04-30 13:00:00,TMP-10.rtf,April 2022,Zaradi pokvarjenega vozila je na štajerski avt...,"Na mejnih prehodih Sečovlje, Petrina, Dragonja...", ,None,None
2,2022-04-27 06:30:00,TMP-100.rtf,April 2022,Zaradi prometne nesreče je na gorenjski avtoce...,Na mariborski vzhodni obvoznici promet zaradi ...,None,None,None


In [171]:
import re
import json

def _clean_text(text: str) -> str:
    # Replace newlines, tabs, and multiple spaces with a single space
    text = re.sub(r'[\n\r\t]+', ' ', text)
    # Remove list markers like '-' and extra spaces around them
    text = re.sub(r'\s*-\s*', ' ', text)
    # Ensure a space follows a period for sentence splitting, unless it's the end of the string
    text = re.sub(r'\.(?=[a-zA-Z])', '. ', text)
    # Consolidate multiple spaces into one
    text = re.sub(r'\s{2,}', ' ', text)
    return text.strip()

DEPENDENT_PHRASES = [
    r'^Nastal je krajši zastoj',
    r'\bčas potovanja se(?:\s+\w+){0,5}?\s+podaljša\b'
    r'^čas vožnje se podaljša',
    #Predor zaradi varnosti občasno zapirajo
    r'^Predor zaradi varnosti občasno zapirajo',
]

def merge_dependent_sentences(sentences, dependent_phrases):
    merged = []
    i = 0
    while i < len(sentences):
        current = sentences[i]
        is_dependent = (
            i > 0 and (
                any(re.match(p, current.lower()) for p in dependent_phrases)
                or len(current) < 85
            )
        )

        if is_dependent and merged:
            # Safe to merge with previous
            merged[-1] += ' ' + current
        else:
            # Either it's the first sentence or not dependent
            merged.append(current)

        i += 1

    return merged


def parse_traffic_report_exclusive(text: str) -> dict:
    json_format = {
        "Delo na cesti": "",
        "Dela": "",
        "Delovne zapore": "",
        "Popolne zapore": "",
        "Zastoji": "", "Ovire": "", "Nesreče": "", "Opozorila": "", "Vreme": "",
        "Zimske razmere": "", "Prireditve": "", "Mejni prehodi": "",
        "Omejitve za tovorna vozila": "", "leftover_html": ""
    }
    results = {key: [] for key in json_format.keys()}

    # 1. Pre-process the text to handle formatting issues
    cleaned_text = _clean_text(text)
    
    # Split text into sentences and filter out empty strings
    sentences = [s.strip() for s in re.split(r'(?<=[.?!])\s+', cleaned_text) if s.strip()]
    sentences = merge_dependent_sentences(sentences, DEPENDENT_PHRASES)
    # 2. Priority-based Classification Rules
    # The script checks categories in this order. The first match wins.
    # `\b` is a word boundary to prevent partial mat
        
    classification_rules = [
        ('Zastoji', [
            r'\bzastoj\w*\b', r'\bgost.*?promet\b', r'\bzgošč.*?promet\b',
            r'čas.*?potovanj.*?podaljš\w*', r'potoval.*?res.*?podaljš\w*',
            r'\bkolon\w*\b', r'\bupoč\w*\b', r'\bzamud\w*\b'
        ]),

        ('Opozorila', [ 
            r'\bopozarj\w*\b', r'\bnevarn\w*\b', r'\bpriporoč\w*\b',
            r'\bpozorn\w*\b', r'\bobvoz\w*\b', r'\bpreusmer\w*\b',
            r'\bpozar\w*\b', r'\bspolz\w*\b', r'\bbo\b', r'\bbodo\b',
            
            # New patterns for extended travel time
            r'\b(potovaln\w*|čas\s+potovanja)\s+se\s+podaljš\w*\b',
            r'\bse\s+podaljš\w*\s+(potovaln\w*|čas\s+potovanja)\b',
            r'\bčas\s+se\s+podaljš\w*\b',
            r'\bpodaljšan\s+(čas|potovaln\w*)\b'
        ]),


        ('Zimske razmere', [
            r'\bsneg\w*\b', r'\bverig\w*\b', r'\bzims\w*\b', r'\bled\w*\b'
        ]),

        ('Vreme', [
            r'\bmegl\w*\b', r'\bpoledic\w*\b', r'\bburj\w*\b', r'\bvidljiv\w*\b'
        ]),

        ('Prireditve', [
            r'\bprired\w*\b', r'\bdogod\w*\b', r'\bmaraton\w*\b'
        ]),

        ('Nesreče', [
            r'\bnesreč\w*\b', r'\btrčen\w*\b', r'\bnezgod\w*\b'
        ]),

        ('Ovire', [
            r'\bovir\w*\b', r'\bpokvarj\w*\s+(voz\w*|tovornjak\w*)\b',
            r'\bobtič\w*\b', r'\brazsut\w*\s+tovor\w*\b'
        ]),

        ('Popolne zapore', [
            r'zap\w*.*?(regionaln\w*\s+cest\w*|avtocest\w*|cest\w*|hitr\w*\s+cest\w*|obvoznic\w*|križišč\w*|predor\w*|uvoz\w*|izvoz\w*)',
            r'\bpopoln\w*\s+zapor\w*\b'
        ]),

        ('Delovne zapore', [
            r'zap\w*.*?(\bvozn\w*\s+pas\w*|\bprehit\w*\s+pas\w*|\bodstav\w*\s+pas\w*|\bpočasn\w*\s+vozil\w*\s+pas\w*)',
            r'zožen\w*\s+pas\w*'
        ]),

        ('Delo na cesti', [
            r'del\w*.*?cest\w*', r'promet.*?(urej\w*|potek\w*)', r'gradbišč\w*', r'delovišč\w*'
        ]),

        ('Omejitve za tovorna vozila', [
            r'prepoved\w*\s+promet\w*\s+za\b', r'tovorn\w*\s+vozil\w*\b', r'tovornjak\w*\b',
            r'hladilnik\w*\b', r'ponjav\w*\b'
        ]),

        ('Mejni prehodi', [
            r'mejn\w*\b', r'carin\w*\b', r'\b(prehod|vstop\w*).*dr\w*\b',  r'\b(prehod|izstop\w*).*dr\w*\b'
        ]),
    ]



    unclassified_sentences = list(sentences)

    for category, patterns in classification_rules:
        remaining_sentences = []

        for sentence in unclassified_sentences:
            matched = False
            for pattern in patterns:
                if re.search(pattern, sentence.lower()):
                    results[category].append(sentence)
                    matched = True
                    break  # Stop checking further patterns for this category

            if not matched:
                remaining_sentences.append(sentence)

        unclassified_sentences = remaining_sentences  # Only keep unmatched for next category



    # 3. Handle leftovers
    results['leftover_html'] = unclassified_sentences

    # 4. Aggregate 'Dela' category
    results['Dela'] = results['Popolne zapore'] + results['Delovne zapore'] + results['Delo na cesti']
    
    # 5. Finalize the JSON by joining sentence lists into strings
    final_json = {}
    for key, sent_list in results.items():
        final_json[key] = " ".join(sent_list)

    return final_json, cleaned_text


def parse_output_messages(output_row, columns):
    row_dict = dict(zip(columns, output_row))
    message_parts = []
    for i in range(1, 6):  # Loop through content_01 to content_05
        col_name = f"content_0{i}"
        content = row_dict.get(col_name)
        if content:
            cleaned = str(content).replace('\x00', '')  # Remove ALL null characters
            if cleaned.strip():  # Only append if there's still visible text
                message_parts.append(cleaned)
    return " ".join(message_parts)


sample_df = output_df.sample(n=1)  # Sample 3 rows for testing

from IPython.display import display, HTML

for i, output_row in enumerate(sample_df.iter_rows(), 1):
    print(f"--- PARSED OUTPUT FOR EXAMPLE {i} ---")
    output_message = parse_output_messages(output_row, output_df.columns)
    parsed_json, cleaned_output_message = parse_traffic_report_exclusive(output_message)
    # Use json.dumps for pretty printing with UTF-8 support
    display(HTML(f"<pre style='white-space: pre-wrap;'>Output Message {i}: {cleaned_output_message}</pre>"))
    print(json.dumps(parsed_json, indent=4, ensure_ascii=False))
    print("\n")


--- PARSED OUTPUT FOR EXAMPLE 1 ---


{
    "Delo na cesti": "",
    "Dela": "",
    "Delovne zapore": "",
    "Popolne zapore": "",
    "Zastoji": "",
    "Ovire": "Zaradi pokvarjenega vozila je na primorski avtocesti proti Kopru promet oviran pred priključkom Kozina.",
    "Nesreče": "Promet na podravski avtocesti proti Gruškovju med priključkoma Lancova vas in Podlehnik ni več oviran zaradi nesreče.",
    "Opozorila": "",
    "Vreme": "",
    "Zimske razmere": "",
    "Prireditve": "",
    "Mejni prehodi": "",
    "Omejitve za tovorna vozila": "",
    "leftover_html": ""
}




In [216]:
#optimizzed version
import pandas as pd
import polars as pl
import json
from tqdm import tqdm
import warnings
from sentence_transformers import SentenceTransformer, util

warnings.filterwarnings("ignore")

# Assume these functions are defined elsewhere in your script
# from your_utils import parse_html_into_json, get_first_inputs_before_time

# It's good practice to load the model once
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')


def compare_json_to_output_optimized(input_df, output_df, n_samples=15):
    """
    Optimized function to find the best input match for each output message.
    """
    best_results_list = []
    output_columns = output_df.columns

    print(f"Comparing {len(output_df)} output examples with {n_samples} input samples each...")

    # Iterate directly over output_df rows. This is much cleaner than sampling in a loop.
    for output_row_tuple in tqdm(output_df.iter_rows(), total=len(output_df), desc="Processing Output Examples"):

        try:
            # --- 1. Process the output ONCE per outer loop ---
            output_message = parse_output_messages(output_row_tuple, output_columns)
            output_json, output_message = parse_traffic_report_exclusive(output_message)
            output_json["Timestamp"] = output_row_tuple[0].strftime('%Y-%m-%d %H:%M:%S')
            output_json_str = json.dumps(output_json, indent=2, ensure_ascii=False)
            if not output_message: # Skip if the output message is empty
                continue
            
            # Encode the single output message
            output_embedding = model.encode(output_message, convert_to_tensor=True)

            # --- 2. Get candidate inputs ---
            input_rows_df = get_first_inputs_before_time(n_samples, output_row_tuple, input_df).to_pandas()
            input_rows_df_timesort = get_first_input_before_time(1, output_row_tuple, input_df).to_pandas()
            # print(f"Input rows for output row {output_row_tuple[0]}: {len(input_rows_df_timesort)} candidates found.")
            
            for _, input_row in input_rows_df_timesort.iterrows():
                html_timesort = str(input_row.get("B1", ""))
                myjson_timesort = parse_html_into_json(html_timesort, json_format, input_row["Datum"]) 
                # print(f"Parsed JSON for timesort: {myjson_timesort}")
            
            
            
            if input_rows_df.empty:
                continue

            # --- 3. Prepare all input texts for BATCH processing ---
            input_texts_to_encode = []
            processed_inputs = [] # Keep track of the full parsed data

            for _, input_row in input_rows_df.iterrows():
                html = str(input_row.get("B1", ""))
                
                # Assume parse_html_into_json is defined and returns a dict
                myjson = parse_html_into_json(html, json_format, input_row["Datum"]) 
                
                full_string = " ".join(value for value in myjson.values() if value)
                input_texts_to_encode.append(full_string)
                processed_inputs.append({
                    "Input_Message": full_string,
                    "Input_JSON_str": json.dumps(myjson, indent=2, ensure_ascii=False),
                    "Datum": input_row["Datum"]
                })

            # --- 4. BATCH encode all input texts at once ---
            input_embeddings = model.encode(input_texts_to_encode, convert_to_tensor=True)
            
            # --- 5. Compute all similarities in a single, fast operation ---
            cosine_scores = util.cos_sim(output_embedding, input_embeddings)
            # print(f"Cosine scores: {cosine_scores}")
            
            # Find the index of the best score
            best_sample_idx = cosine_scores[0].argmax().item()

            # --- 6. Select the best result ---
            best_input_data = processed_inputs[best_sample_idx]
            best_score = cosine_scores[0][best_sample_idx].item()
            # print(f"Best score for output row {output_row_tuple[0]}: {best_score:.4f}")
            
            # Append the single best result for this output to our list
            best_results_list.append({
                "Input_Message": best_input_data["Input_Message"],
                "Input_JSON": best_input_data["Input_JSON_str"],
                "Output_Message": output_message,
                "Output_JSON": output_json_str,
                "Datum": best_input_data["Datum"],
                "Similarity_Score": best_score,
                "Input_timesort": html_timesort,
                "json_timesort": json.dumps(myjson_timesort, indent=2, ensure_ascii=False),
            })

        except Exception as e:
            # It's good to know which row caused an error
            output_timestamp = output_row_tuple[0] 
            print(f"Error processing output row for timestamp {output_timestamp}: {e}")
            continue

    # --- 7. Create the final DataFrame ONCE from the list of results ---
    if not best_results_list:
        print("No valid examples were processed.")
        return pd.DataFrame()
        
    final_df = pd.DataFrame(best_results_list)
    return final_df

# Assuming input_df and output_df are already loaded as Polars DataFrames
# And json_format is defined
#sample the outputdf  
output_df_sample = output_df.sample(n=1)  # Sample 15 rows for testing
traffic_examples = compare_json_to_output_optimized(input_df, output_df, n_samples=10)
traffic_examples.to_csv("trainingdataset_optimized.csv", index=False, encoding="utf-8")


def display_jsons_side_by_side(input_json, output_json, json_timesort):
    input_pretty = json.dumps(input_json, indent=2, ensure_ascii=False)
    output_pretty = json.dumps(output_json, indent=2, ensure_ascii=False)
    timesort_pretty = json.dumps(json_timesort, indent=2, ensure_ascii=False)
    
    html = f"""
    <div style="display: flex; gap: 20px; font-family: monospace;">
        <div style="flex: 1; white-space: pre; border: 1px solid #ccc; padding: 10px; overflow: auto; max-height: 400px;">
            <h3>Input JSON</h3>
            <pre>{input_pretty}</pre>
        </div>
        <div style="flex: 1; white-space: pre; border: 1px solid #ccc; padding: 10px; overflow: auto; max-height: 400px;">
            <h3>Output JSON</h3>
            <pre>{output_pretty}</pre>
        </div>
    </div>
        <div style="display: flex; gap: 20px; font-family: monospace;">
        <div style="flex: 1; white-space: pre; border: 1px solid #ccc; padding: 10px; overflow: auto; max-height: 400px;">
            <h3>Timesort JSON</h3>
            <pre>{input_pretty}</pre>
        </div>
    </div>
    """
    display(HTML(html))
    
# for i, row in traffic_examples.iterrows():
#     print(f"--- Example {i+1} ---")
#     input_json = json.loads(row["Input_JSON"])
#     output_json = json.loads(row["Output_JSON"])
#     json_timesort = json.loads(row["json_timesort"])
    
    
#     display(HTML(f"<pre style='white-space: pre-wrap;'>Original Timesort Message : {row["Input_timesort"]}</pre>"))
#     display(HTML(f"<pre style='white-space: pre-wrap;'>Input Semanticsort Message : {row["Input_Message"]}</pre>"))
#     display(HTML(f"<pre style='white-space: pre-wrap;'>Output Message : {row["Output_Message"]}</pre>"))
#     display_jsons_side_by_side(input_json, output_json, json_timesort)
    
#     print(f"Similarity Score: {row['Similarity_Score']:.4f}")
#     print("-" * 80)


Comparing 23617 output examples with 10 input samples each...


Processing Output Examples:   0%|          | 29/23617 [00:06<55:59,  7.02it/s]  

Error processing output row for timestamp 2022-04-26 08:00:00: 'NoneType' object has no attribute 'to_pandas'


Processing Output Examples:   5%|▌         | 1223/23617 [05:00<1:07:46,  5.51it/s]

Error processing output row for timestamp 2022-08-10 20:00:00: 'NoneType' object has no attribute 'to_pandas'


...

Processing Output Examples:  93%|█████████▎| 21994/23617 [1:26:52<03:51,  7.00it/s]

Error processing output row for timestamp 2024-11-24 10:30:00: 'NoneType' object has no attribute 'to_pandas'


Processing Output Examples:  96%|█████████▌| 22702/23617 [1:29:45<02:18,  6.61it/s]

Error processing output row for timestamp 2024-10-21 18:00:00: 'NoneType' object has no attribute 'to_pandas'
Error processing output row for timestamp 2024-10-21 17:30:00: 'NoneType' object has no attribute 'to_pandas'
Error processing output row for timestamp 2024-10-21 17:00:00: 'NoneType' object has no attribute 'to_pandas'


Processing Output Examples: 100%|██████████| 23617/23617 [1:32:08<00:00,  4.27it/s]


In [211]:
import re
import json
from sentence_transformers import SentenceTransformer, util

# It's best to load the model only once
# 'paraphrase-multilingual-MiniLM-L12-v2' is a good, lightweight model for many languages.
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

def get_all_text_from_json(data: dict, exclude_keys=None) -> list:
    """Extracts all sentences from the text fields of the JSON."""
    if exclude_keys is None:
        exclude_keys = ["timestamp", "Dela"] # Exclude timestamp and aggregated 'Dela'
        
    all_sentences = []
    for key, value in data.items():
        if key.lower() in exclude_keys or not isinstance(value, str) or not value:
            continue
        # Split value into sentences, handling cases where there are multiple sentences in one field
        sentences = re.split(r'(?<=[. ?!])\s+', value.strip())
        for sentence in sentences:
            if sentence:
                # Store sentence with its original category
                all_sentences.append({'sentence': sentence, 'category': key})
    return all_sentences


def filter_input_by_output_relevance(input_json: dict, output_json: dict, similarity_threshold: float = 0.5):
    """
    Filters the input_json to only include sentences that are semantically
    relevant to the text in the output_json.

    Args:
        input_json: The original, noisy input data.
        output_json: The ground truth broadcasted output.
        similarity_threshold: The cosine similarity score required to be considered 'relevant'.

    Returns:
        A new, cleaned input JSON containing only relevant information.
    """
    # 1. Extract all sentences from input and all text from output
    input_sentences_with_meta = get_all_text_from_json(input_json)
    output_sentences_with_meta = get_all_text_from_json(output_json)
    
    # If there's nothing in the output, we can't determine relevance.
    if not output_sentences_with_meta:
        # Return a mostly empty structure matching the input
        clean_input = {k: "" for k in input_json}
        clean_input['timestamp'] = input_json.get('timestamp', '')
        return clean_input

    # Combine all output sentences into one target script for a single comparison point
    target_script = " ".join([item['sentence'] for item in output_sentences_with_meta])
    
    # Extract just the sentence text for encoding
    input_sentence_texts = [item['sentence'] for item in input_sentences_with_meta]

    # 2. Compute embeddings
    target_embedding = model.encode(target_script, convert_to_tensor=True)
    input_embeddings = model.encode(input_sentence_texts, convert_to_tensor=True)

    # 3. Calculate cosine similarity
    cosine_scores = util.cos_sim(target_embedding, input_embeddings)

    # 4. Filter relevant sentences
    relevant_sentences_with_meta = []
    for i, item in enumerate(input_sentences_with_meta):
        if cosine_scores[0][i] > similarity_threshold:
            relevant_sentences_with_meta.append(item)
            print(f"MATCH (Score: {cosine_scores[0][i]:.2f}): {item['sentence']}")
        else:
            print(f"NO MATCH (Score: {cosine_scores[0][i]:.2f}): {item['sentence']}")


    # 5. Reconstruct the new, clean input JSON
    # Start with an empty structure
    clean_input_json = {key: [] for key in input_json.keys()}

    for item in relevant_sentences_with_meta:
        clean_input_json[item['category']].append(item['sentence'])
    
    # Join the lists of sentences back into strings
    for key, value_list in clean_input_json.items():
        clean_input_json[key] = " ".join(value_list)
        
    # Re-aggregate the 'Dela' field from its constituent parts
    dela_parts = [
        clean_input_json.get('Popolne zapore', ''),
        clean_input_json.get('Delovne zapore', ''),
        clean_input_json.get('Delo na cesti', '')
    ]
    clean_input_json['Dela'] = " ".join(part for part in dela_parts if part)


    # Preserve the original timestamp
    clean_input_json['timestamp'] = input_json.get('timestamp')
    
    return clean_input_json

# --- Your Example Data ---
input_json = {
  "Delo na cesti": "",
  "Dela": "Na gorenjski avtocesti bo promet skozi predor Karavanke urejen izmenično enosmerno s čakanjem pred predorom 30 minut, med 8. in 16. uro: 2., 3., 6., 8., 9. in 10. marca. Na primorski avtocesti bo od jutri, predvidoma do konca marca, promet med razcepom Srmin in priključkom Črni Kal potekal po enem pasu v vsako smer. Zaprt bo razcep Srmin iz smeri Škofij proti Ljubljani ter priključek Luka Koper. Obvoz iz smeri Škofij proti Ljubljani bo preko priključka Bertoki.",
  "Delovne zapore": "",
  "Popolne zapore": "",
  "Zastoji": "Zaradi jutranje prometne konice je promet povečan na mestnih obvoznicah in vpadnicah.",
  "Ovire": "Na štajerski avtocesti je zaradi okvare tovornega vozila oviran promet med priključkoma Slovenska Bistrica jug in Slovenske Konjice proti Ljubljani.",
  "Nesreče": "Na ljubljanski zahodni obvoznici je oviran promet na uvozu Brdo proti Kosezam. Nastal je zastoj do razcepa Kozarje.",
  "Vreme": "Cesta čez prelaz Vršič je prevozna samo za osebna vozila.",
  "Opozorila": "",
  "Zimske razmere": "",
  "Prireditve": "",
  "Mejni prehodi": "",
  "Omejitve za tovorna vozila": "",
  "leftover_html": "",
  "timestamp": "02-03-2022 07:15:00"
}

output_json = {
  "Delo na cesti": "",
  "Dela": "",
  "Delovne zapore": "",
  "Popolne zapore": "",
  "Zastoji": "",
  "Ovire": "",
  "Nesreče": "",
  "Opozorila": "Na primorski avtocesti bo od danes zaradi del med razcepom Srmin in priključkom Črni Kal promet v obe smeri potekal le po enem prometnem pasu. Razcep Srmin bo iz smeri Škofij zaprt proti Ljubljani. Obvoz bo prek priključka Bertoki. Zaprt bo tudi priključek Luka Koper proti Ljubljani.",
  "Vreme": "",
  "Zimske razmere": "",
  "Prireditve": "",
  "Mejni prehodi": "",
  "Omejitve za tovorna vozila": "",
  "leftover_html": "Cesta čez prelaz Vršič je prevozna samo za osebna vozila.",
  "Timestamp": "2022-03-02 07:00:00"
}

# --- Run the filter ---
print("--- Determining Relevance ---")
clean_training_input = filter_input_by_output_relevance(input_json, output_json, similarity_threshold=0.6)

print("\n--- Original Input ---")
print(json.dumps(input_json, indent=2, ensure_ascii=False))

print("\n--- NEW, CLEANED Input for Training ---")
print(json.dumps(clean_training_input, indent=2, ensure_ascii=False))

--- Determining Relevance ---
MATCH (Score: 0.62): Na gorenjski avtocesti bo promet skozi predor Karavanke urejen izmenično enosmerno s čakanjem pred predorom 30 minut, med 8.
NO MATCH (Score: 0.05): in 16.
NO MATCH (Score: 0.07): uro: 2., 3., 6., 8., 9.
NO MATCH (Score: 0.01): in 10.
NO MATCH (Score: 0.13): marca.
MATCH (Score: 0.74): Na primorski avtocesti bo od jutri, predvidoma do konca marca, promet med razcepom Srmin in priključkom Črni Kal potekal po enem pasu v vsako smer.
MATCH (Score: 0.76): Zaprt bo razcep Srmin iz smeri Škofij proti Ljubljani ter priključek Luka Koper.
MATCH (Score: 0.77): Obvoz iz smeri Škofij proti Ljubljani bo preko priključka Bertoki.
NO MATCH (Score: 0.32): Zaradi jutranje prometne konice je promet povečan na mestnih obvoznicah in vpadnicah.
MATCH (Score: 0.75): Na štajerski avtocesti je zaradi okvare tovornega vozila oviran promet med priključkoma Slovenska Bistrica jug in Slovenske Konjice proti Ljubljani.
NO MATCH (Score: 0.45): Na ljubljanski zahod